In [20]:
import pandas as pd
import json
import os
from os import listdir
from os.path import isfile, join
import ast
import numpy as np
import stanza
import re
#stanza.download('da')

In [2]:
import_cols = ['created_at', 'id', 'full_text', 'entities', 'user', 'retweeted_status', 'is_quote_status', 'retweet_count', 
            'favorite_count', 'favorited', 'retweeted']

user_infos = ['id', 'name', 'screen_name', 'location', 'description', 'url', 'followers_count', 'created_at', 'verified']

keep_cols = ['created_at', 'id', 'full_text', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',
            'is_retweet', 'hashtags', 'user_mentions', 'urls'] + ["user_" + user_info for user_info in user_infos]

#savepath = os.path.join('C:/', 'data', 'poltweets', "tweets_flattened_20200127.gz")
datapath = os.path.join('C:/', 'data', 'poltweets', "tweets_flattened_20200127.gz")

In [3]:
nlp = stanza.Pipeline('da', processors = 'tokenize,pos') # no sentiment analysis for Danish

2021-01-27 15:32:22 INFO: Loading these models for language: da (Danish):
| Processor | Package |
-----------------------
| tokenize  | ddt     |
| pos       | ddt     |

2021-01-27 15:32:22 INFO: Use device: cpu
2021-01-27 15:32:22 INFO: Loading: tokenize
2021-01-27 15:32:22 INFO: Loading: pos
2021-01-27 15:32:23 INFO: Done loading processors!


In [124]:
# Load data
data = pd.read_csv(datapath, index_col = 0)

In [126]:
sample = data.loc[data['is_retweet'] == False,:].sample(n = 10000, random_state = 42).reset_index(drop = True)

In [127]:
def climate_match(string):
    regex = re.compile(r'.*klima.*|.*miljø.*', re.IGNORECASE)
    
    if regex.match(string):
        return True
    else:
        return False

sample['talk_climate'] = sample['full_text'].apply(lambda x: climate_match(x))

In [128]:
df = sample.loc[sample['talk_climate'], :].reset_index(drop = True)

In [129]:
df.shape

(303, 22)

In [130]:
df.head()

,created_at,id,full_text,is_quote_status,retweet_count,favorite_count,favorited,retweeted,is_retweet,hashtags,...,user_id,user_name,user_screen_name,user_location,user_description,user_url,user_followers_count,user_created_at,user_verified,talk_climate
0,Thu Nov 22 09:57:49 +0000 2018,1065544874873823200,Nej Mette F - debatten om kød er ikke hysteris...,False,13,65,False,False,False,"['dkpol', 'dkgreen', 'derforuffe']",...,1127968447,rasmus nordqvist 🏳️‍🌈,rasmusnordqvist,Copenhagen,"Natur, landbrug & fiskeri, Int. udvikling, med...",https://t.co/Rb6deTVKUq,9926,Mon Jan 28 12:47:09 +0000 2013,True,True
1,Sun Dec 22 09:46:56 +0000 2019,1208685341315682300,Ja tak - klimaafgifter ligger lige for også på...,True,7,45,False,False,False,"['dkpol', 'dkklima']",...,1127968447,rasmus nordqvist 🏳️‍🌈,rasmusnordqvist,Copenhagen,"Natur, landbrug & fiskeri, Int. udvikling, med...",https://t.co/Rb6deTVKUq,9926,Mon Jan 28 12:47:09 +0000 2013,True,True
2,Tue Feb 26 21:29:22 +0000 2019,1100508144311128000,"@larsloekke sagde i 2014 at det var både ""urea...",False,2,10,False,False,False,"['dkpol', 'dkgreen']",...,266667139,Jens Joel,Jens_Joel,NaN,"Folketingsmedlem (S). Bor i Aarhus, valgt i Øs...",https://t.co/YxDz9dEgjd,7295,Tue Mar 15 16:10:47 +0000 2011,False,True
3,Sat Jun 06 21:42:10 +0000 2020,1269384108423155700,"Det er som om nogle tror, at de menneskeskabte...",True,15,179,False,False,False,['dkpol'],...,2373406198,Jacob Mark,jacobmark_sf,NaN,Gruppeformand for SF. Børne- og undervisningso...,NaN,12946,Sat Mar 01 16:04:14 +0000 2014,True,True
4,Wed Jun 27 07:53:22 +0000 2018,1011880149313302500,@TheDorteOlsen @nielscallesoe @MonbergSF @Fabu...,False,0,1,False,False,False,[],...,2304249887,Karsten Hønge,khoenge,NaN,Verbal nævefighter. Politik skal ikke være unø...,https://t.co/ovpq2ryxpX,7492,Sat Jan 25 20:35:49 +0000 2014,True,True


In [131]:
def tokenizer_custom(text):
    doc = nlp(text)
    
    tag_match = re.compile(r'(?!^\@|^\#)')
    
    pos_tags = ['ADJ', 'NOUN'] # Keeps proper nouns, adjectives and nouns
    
    tokens = []
      
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.pos in pos_tags:
                token = word.text.lower() # Returning the lemma of the word in lower-case.
                tokens.append(token)
    
    tokens = list(filter(tag_match.match, tokens))
    
    return(tokens)

In [132]:
df['tokens'] = df['full_text'].apply(tokenizer_custom)

**To do**

- Variabel for partitilhør
- Behandling af datovariabel
- Opgørelse: klimatweets over tid?
- Hvilke partier tweeter mest om klima?
- Mest brugte ord i forbindelse med klima?
    - Udvikling over tid
    - Visualisering? - hvordan? (ordsky, netværk)
- Engagement-variabel (retweet + favorites)
- Inklusion af bestemte ords betydning for engagemenet?